# AR6 calibration of FaIR 2.1

Calibration
- use the exact aerosol indirect forcing relationship from Smith et al. (2021). 
- aerosol direct is from AR6.
- three layer model for climate response.
- include overlap of the major GHGs.
- prognostic equation for land use related forcing (e.g. from FaIR 1.6).
- ozone relationship from FaIR 1.6 used in AR6.
- interactive methane lifetime (NEW!)

TODO:
- solar trend
- vary pre-industrial concentrations of CO2

## Basic imports

In [ ]:
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats
from tqdm import tqdm

from fair21 import (
    SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, RunConfig, CH4LifetimeMethod, FAIR
)
from fair21.defaults import species_config_from_default

## Set up problem

In [ ]:
species_ids = {
    # Greenhouse gases and precursors
    'CO2_FFI': SpeciesID('CO2 fossil fuel and industrial', Category.CO2_FFI),
    'CO2_AFOLU': SpeciesID('CO2 AFOLU', Category.CO2_AFOLU),
    'CO2': SpeciesID('CO2', Category.CO2),
    'CH4': SpeciesID('CH4', Category.CH4),
    'N2O': SpeciesID('N2O', Category.N2O),
    'CFC-11': SpeciesID('CFC-11', Category.CFC_11),
    'CFC-12': SpeciesID('CFC-12', Category.OTHER_HALOGEN),
    'CFC-113': SpeciesID('CFC-113', Category.OTHER_HALOGEN),
    'CFC-114': SpeciesID('CFC-114', Category.OTHER_HALOGEN),
    'CFC-115': SpeciesID('CFC-115', Category.OTHER_HALOGEN),
    'HCFC-22': SpeciesID('HCFC-22', Category.OTHER_HALOGEN),
    'HCFC-141b': SpeciesID('HCFC-141b', Category.OTHER_HALOGEN),
    'HCFC-142b': SpeciesID('HCFC-142b', Category.OTHER_HALOGEN),
    'CCl4': SpeciesID('CCl4', Category.OTHER_HALOGEN),
    'CHCl3': SpeciesID('CHCl3', Category.OTHER_HALOGEN),
    'CH2Cl2': SpeciesID('CH2Cl2', Category.OTHER_HALOGEN),
    'CH3Cl': SpeciesID('CH3Cl', Category.OTHER_HALOGEN),
    'CH3CCl3': SpeciesID('CH3CCl3', Category.OTHER_HALOGEN),
    'CH3Br': SpeciesID('CH3Br', Category.OTHER_HALOGEN),
    'Halon-1211': SpeciesID('Halon-1211', Category.OTHER_HALOGEN),
    'Halon-1301': SpeciesID('Halon-1301', Category.OTHER_HALOGEN),
    'Halon-2402': SpeciesID('Halon-2402', Category.OTHER_HALOGEN),
    'CF4': SpeciesID('CF4', Category.F_GAS),
    'C2F6': SpeciesID('C2F6', Category.F_GAS),
    'C3F8': SpeciesID('C3F8', Category.F_GAS),
    'c-C4F8': SpeciesID('C-C4F8', Category.F_GAS),
    'C4F10': SpeciesID('C4F10', Category.F_GAS),
    'C5F12': SpeciesID('C5F12', Category.F_GAS),
    'C6F14': SpeciesID('C6F14', Category.F_GAS),
    'C7F16': SpeciesID('C7F16', Category.F_GAS),
    'C8F18': SpeciesID('C8F18', Category.F_GAS),
    'HFC-125': SpeciesID('HFC-125', Category.F_GAS),
    'HFC-134a': SpeciesID('HFC-134a', Category.F_GAS),
    'HFC-143a': SpeciesID('HFC-143a', Category.F_GAS),
    'HFC-152a': SpeciesID('HFC-152a', Category.F_GAS),
    'HFC-227ea': SpeciesID('HFC-227ea', Category.F_GAS),
    'HFC-23': SpeciesID('HFC-23', Category.F_GAS),
    'HFC-236fa': SpeciesID('HFC-236fa', Category.F_GAS),
    'HFC-245fa': SpeciesID('HFC-245fa', Category.F_GAS),
    'HFC-32': SpeciesID('HFC-32', Category.F_GAS),
    'HFC-365mfc': SpeciesID('HFC-365mfc', Category.F_GAS),
    'HFC-4310mee': SpeciesID('HFC-4310mee', Category.F_GAS),
    'NF3': SpeciesID('NF3', Category.F_GAS),
    'SF6': SpeciesID('SF6', Category.F_GAS),
    'SO2F2': SpeciesID('SO2F2', Category.F_GAS),
    # aerosols, ozone, and their precursors
    'Sulfur': SpeciesID('Sulfur', Category.SULFUR),
    'BC': SpeciesID('BC', Category.BC),
    'OC': SpeciesID('OC', Category.OC),
    'NH3': SpeciesID('NH3', Category.OTHER_AEROSOL),
    'VOC': SpeciesID('VOC', Category.REACTIVE_GAS),
    'CO': SpeciesID('CO', Category.REACTIVE_GAS),
    'NOx': SpeciesID('NOx', Category.REACTIVE_GAS),
    'ari': SpeciesID('Aerosol-Radiation Interactions', Category.AEROSOL_RADIATION_INTERACTIONS),
    'aci': SpeciesID('Aerosol-Cloud Interactions', Category.AEROSOL_CLOUD_INTERACTIONS),
    'ozone': SpeciesID('Ozone', Category.OZONE),
    # Contrails and precursors
    'NOx_aviation': SpeciesID('NOx Aviation', Category.NOX_AVIATION),
    'contrails': SpeciesID('Contrails', Category.CONTRAILS),
    # other minor anthropogenic
    'LAPSI': SpeciesID('Light absorbing particles on snow and ice', Category.LAPSI),
    'H2O_stratospheric': SpeciesID('H2O Stratospheric', Category.H2O_STRATOSPHERIC),
    'land_use': SpeciesID('Land Use', Category.LAND_USE),
    # natural
    'solar': SpeciesID('Solar', Category.SOLAR),
    'volcanic': SpeciesID('Volcanic', Category.VOLCANIC)
}

In [ ]:
scenarios_to_include=['ssp245']

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
df_forc = pd.read_csv('../data/forcing/table_A3.3_historical_ERF_1750-2019_best_estimate.csv')

In [ ]:
emitted_species = [
    'CO2_FFI', 'CO2_AFOLU', 'CH4', 'N2O',
    'Sulfur', 'BC', 'OC', 'NH3', 'NOx', 'VOC', 'CO',
    'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115',
    'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br',
    'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18',
    'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32', 
    'HFC-365mfc', 'HFC-4310mee', 'NOx_aviation']
forced_species = ['solar', 'volcanic']
from_other_species = ['CO2', 'ari', 'aci', 'ozone', 'contrails', 'LAPSI', 'H2O_stratospheric', 'land_use']

species_to_include = emitted_species + forced_species + from_other_species

In [ ]:
scenarios = []
for iscen, scenario in enumerate(scenarios_to_include):
    list_of_species = []
    for ispec, species in enumerate(emitted_species):
        species_rcmip_name = species.replace("-", "")
        if species == 'NOx_aviation':
            species_rcmip_name = 'NOx|MAGICC Fossil and Industrial|Aircraft'
        elif species == 'CO2_FFI':
            species_rcmip_name = 'CO2|MAGICC Fossil and Industrial'
        elif species == 'CO2_AFOLU':
            species_rcmip_name = 'CO2|MAGICC AFOLU'
        emis_in = df_emis.loc[
            (df_emis['Scenario']==scenario) & (df_emis['Variable'].str.endswith("|"+species_rcmip_name)) & 
            (df_emis['Region']=='World'), '1750':'2030'
        ].interpolate(axis=1).values.squeeze()

        # CO2 and N2O units need to behave: TODO, sort this out
        if species in ('CO2_FFI', 'CO2_AFOLU', 'N2O'):
            emis_in = emis_in / 1000
        list_of_species.append(Species(species_ids[species], emissions=emis_in))
        #print(species_rcmip_name, emis_in.shape)
        
    # solar and volcanic forcing still a little hacky
    solar_forcing = np.zeros(281)
    solar_forcing[:270] = df_forc['solar'].values
    volcanic_forcing = np.zeros(281)
    volcanic_forcing[:270] = df_forc['volcanic'].values
    volcanic_forcing[269:] = np.linspace(1, 0, 12) * volcanic_forcing[269]
    list_of_species.append(Species(species_ids['solar'], forcing=solar_forcing))
    list_of_species.append(Species(species_ids['volcanic'], forcing=volcanic_forcing))
    
    # add derived species: at this stage just a declaration that we want them
    for species in from_other_species:
        list_of_species.append(Species(species_ids[species]))
        
    scenarios.append(Scenario(scenario, list_of_species))

In [ ]:
samples = 2000
batch_size = 2000

## Load in pre-calculated parameter sets

In [ ]:
df_cc=pd.read_csv('../data/parameter_sets/carbon_cycle.csv')
df_cr=pd.read_csv('../data/parameter_sets/climate_response.csv')
df_aci=pd.read_csv('../data/parameter_sets/erfaci.csv')
df_ari=pd.read_csv('../data/parameter_sets/erfari.csv')
df_ozone=pd.read_csv('../data/parameter_sets/ozone.csv')
df_scaling=pd.read_csv('../data/parameter_sets/forcing_scaling.csv')

In [ ]:
df_ari

In [ ]:
df_aci

In [ ]:
df_cc

In [ ]:
df_cr

In [ ]:
df_scaling

In [ ]:
species_index = {}
for i in range(len(species_to_include)):
    species_index[species_to_include[i]] = i

In [ ]:
for ind in df_scaling:
    if ind in ['minorGHG', 'solar_amplitude', 'solar_trend']:
        continue
    print(ind, species_index[ind])


species_index

## Generate 1 million ensemble members in batches of 1000

## Fill in the Configs

- Grab ClimateResponse configs from pre-calculated ensemble
- SpeciesConfigs will require careful handling

In [ ]:
seedgen = 1355763
run_config = RunConfig(ch4_lifetime_method=CH4LifetimeMethod.AERCHEMMIP)

# for all except temperature, the full time series is not important.
temp_out = np.ones((181, samples)) * np.nan
ohc_out = np.ones((48, samples)) * np.nan
faer_out = np.ones((20, samples)) * np.nan
fari_out = np.ones((270, samples)) * np.nan
faci_out = np.ones((270, samples)) * np.nan
co2_out = np.ones((samples)) * np.nan

for ibatch, batch_start in tqdm(enumerate(range(0, samples, batch_size)), total=samples/batch_size):
    batch_end = batch_start + batch_size
    configs = []
    for iconf in range(batch_start, batch_end):
        config_name = f"{iconf}"

        # Climate response configs
        climate_response = ClimateResponse(
            ocean_heat_capacity = df_cr.loc[iconf, 'c1':'c3'].values.squeeze(),
            ocean_heat_transfer = df_cr.loc[iconf, 'kappa1':'kappa3'].values.squeeze(),
            deep_ocean_efficacy = df_cr.loc[iconf, 'epsilon'],
            gamma_autocorrelation = df_cr.loc[iconf, 'gamma'],
            sigma_eta = df_cr.loc[iconf, 'sigma_eta'],
            sigma_xi = df_cr.loc[iconf, 'sigma_xi'],
            stochastic_run = False,
            seed = seedgen
        )
        seedgen = seedgen+399

        # Species configs
        # get defaults
        species_config = [copy.copy(species_config_from_default(species)) for species in species_to_include]

        # CO2 carbon cycle sensitivity
        species_config[54].iirf_0 = df_cc.loc[iconf, 'r0']
        species_config[54].iirf_airborne = df_cc.loc[iconf, 'rA']
        species_config[54].iirf_cumulative = df_cc.loc[iconf, 'rC']
        species_config[54].iirf_temperature = df_cc.loc[iconf, 'rT']

        # aerosol indirect params
        species_config[56].aci_params = {
            "scale": df_aci.loc[iconf, "beta"],
            "Sulfur": df_aci.loc[iconf, "shape_so2"], 
            "BC+OC": df_aci.loc[iconf, "shape_bcoc"]
        }

        # methane lifetime params (not varied, defaults are baked in)
        species_config[2].lifetime = 10.788405534387858
        species_config[2].natural_emissions_adjustment = 19.019783117809567
        species_config[2].soil_lifetime = 185

        # aerosol direct params
        for index in df_ari:
            species_config[species_index[index]].erfari_radiative_efficiency = df_ari.loc[iconf, index]

        # forcing scalings!
        for index in df_scaling:
            if index in ['minorGHG', 'solar_amplitude', 'solar_trend']:
                continue
            species_config[species_index[index]].scale = df_scaling.loc[iconf, index]
        species_config[52].scale = df_scaling.loc[iconf, 'solar_amplitude']
        for idx in range(11, 51):
            species_config[idx].scale = df_scaling.loc[iconf, 'minorGHG']

        # ozone scalings
        for index in df_ozone:
            if index=='ODS':
                for idx in range(11, 51):
                    species_config[idx].ozone_radiative_efficiency = df_ozone.loc[iconf, 'ODS']
            else:
                species_config[species_index[index]].ozone_radiative_efficiency = df_ozone.loc[iconf, index]
            
        # volcanic efficacy is something like 0.6
        species_config[53].efficacy = 0.6

        configs.append(Config(config_name, climate_response, species_config))
        
    fair = FAIR(scenarios, configs, run_config=run_config)
    fair.run()
    
    # at this point dump out some batch output and put the constraining in another sheet
    temp_out[:, batch_start:batch_end] = fair.temperature[100:, 0, :, 0, 0]
    fair.calculate_ocean_heat_content_change()
    ohc_out[:, batch_start:batch_end] = fair.ocean_heat_content_change[221:269, 0, :, 0, 0]
    co2_out[batch_start:batch_end] = fair.concentration_array[264, 0, :, 54, 0]
    fari_out[:, batch_start:batch_end] = fair.forcing_array[:270, 0, :, 55, 0]
    faci_out[:, batch_start:batch_end] = fair.forcing_array[:270, 0, :, 56, 0]
    faer_out[:, batch_start:batch_end] = fair.forcing_array[250:270, 0, :, 55:57, 0].sum(axis=2)

In [ ]:
# np.save('../data/ar6_ensemble_batches/temperature_1850-2030_20220415.npy', temp_out, allow_pickle=True)
# np.save('../data/ar6_ensemble_batches/ohc_1971-2018_20220415.npy', ohc_out, allow_pickle=True)
# np.save('../data/ar6_ensemble_batches/co2_2014_20220415.npy', co2_out, allow_pickle=True)
# np.save('../data/ar6_ensemble_batches/faer_2000-2019_20220415.npy', faer_out, allow_pickle=True)
# np.save('../data/ar6_ensemble_batches/fari_1750-2019_20220415.npy', fari_out, allow_pickle=True)
# np.save('../data/ar6_ensemble_batches/faci_1750-2019_20220415.npy', faci_out, allow_pickle=True)

In [ ]:
pl.fill_between(np.arange(1750.5,2020), np.percentile(
    fair.forcing_array[:270, 0, :, 57, 0],
    (5), axis=1),
 np.percentile(
    fair.forcing_array[:270, 0, :, 57, 0],
    (95), axis=1)
)
pl.plot(np.arange(1750,2020), np.median(fair.forcing_array[:270, 0, :, 57, 0],axis=1), color='k')

In [ ]:
pl.fill_between(np.arange(1750.5,2020), np.percentile(
    fair.forcing_array[:270, 0, :, 58, 0],
    (5), axis=1),
 np.percentile(
    fair.forcing_array[:270, 0, :, 58, 0],
    (95), axis=1)
)
pl.plot(np.arange(1750,2020), np.median(fair.forcing_array[:270, 0, :, 58, 0],axis=1), color='k')

In [ ]:
pl.fill_between(np.arange(1750.5,2020), np.percentile(
    fair.forcing_array[:270, 0, :, 59, 0],
    (5), axis=1),
 np.percentile(
    fair.forcing_array[:270, 0, :, 59, 0],
    (95), axis=1)
)
pl.plot(np.arange(1750,2020), np.median(fair.forcing_array[:270, 0, :, 59, 0],axis=1), color='k')

In [ ]:
pl.fill_between(np.arange(1750.5,2020), np.percentile(
    fair.forcing_array[:270, 0, :, 60, 0],
    (5), axis=1),
 np.percentile(
    fair.forcing_array[:270, 0, :, 60, 0],
    (95), axis=1)
)
pl.plot(np.arange(1750,2020), np.median(fair.forcing_array[:270, 0, :, 60, 0],axis=1), color='k')

In [ ]:
pl.fill_between(np.arange(1750.5,2020), np.percentile(
    fair.forcing_array[:270, 0, :, 61, 0],
    (5), axis=1),
 np.percentile(
    fair.forcing_array[:270, 0, :, 61, 0],
    (95), axis=1)
)
pl.plot(np.arange(1750,2020), np.median(fair.forcing_array[:270, 0, :, 61, 0],axis=1), color='k')